In [1]:
# import modules
import numpy as np
import pandas as pd
import random
import statsmodels.api as sm

In [2]:
#functions to use

#build OLS model
def fit_linear_reg(y_df,x_df,intercept):
    if intercept:
        mod = sm.OLS(y_df, sm.add_constant(x_df))
    else:
        mod = sm.OLS(y_df, x_df)
    res = mod.fit()
    return res


def nonparm_bootstrapping(file_name,method,y,n_bootstraps,intercept):
    #import
    df=pd.read_csv(file_name)
    n=df.shape[0]
    y_df=df[y]

    #initialize dataframes
    bootstrap_estimates=pd.DataFrame()
    bootstrap_sample=df.iloc[:,1:5] #identifier vars

    if method!='OLS':
        estimates=pd.read_excel('%s.xlsx'%(method),sheet_name='%s%s'%(
            y[0],'_Int' if intercept else ''
        ))
        estimates['Parameter Estimate']=estimates['Parameter Estimate'].astype('float32')
        estimates['Significant Correlate']=estimates['Significant Correlate'].str.replace('*','').str.strip()
        #get only significant correlates
        #estimates=estimates[estimates['Significant Correlate'].str.contains('\*')].copy()
        head=estimates.loc[~estimates['Significant Correlate'].str.contains('Intercept'),'Significant Correlate']
        estimates_dict=dict(zip(estimates['Significant Correlate'],estimates['Parameter Estimate']))
        neighborhood_vars=['AQI','Hosbeds','WindSpeed','RH']
        x=list(estimates_dict.keys())
        x=[item for item in x if 'Intercept' not in item]
        x_df=df[x]
        bootstrap_sample[x]=x_df
        bootstrap_sample[y]=y_df
        # initial yhat
        yhat=0
        for col in x:
            yhat+=x_df[col]*estimates_dict[col]
        if intercept:
            yhat+=sum([value for key, value in estimates_dict.items() if 'Intercept' in key])

        bootstrap_sample['yhat']=yhat
        bootstrap_sample['resid']=bootstrap_sample[y[0]]-bootstrap_sample['yhat']
        resids=bootstrap_sample['resid']

    else:
        x_df=df[x]
        bootstrap_sample[x]=x_df
        bootstrap_sample[y]=y_df
        res=fit_linear_reg(y_df,x_df,intercept)
        base_coeffs=res.params.reset_index().transpose()
        head=base_coeffs.iloc[0]

        #get initial resids
        bootstrap_sample['yhat']=res.fittedvalues
        bootstrap_sample['resid']=res.resid
        resids=res.resid

    #create bootstrap samples from resids
    if method=='STEM':
        neighbor_df=df[neighborhood_vars]
        neighborhood_estimates_dict={}
        for var in estimates_dict.keys():
            if var in neighborhood_vars:
                neighborhood_estimates_dict[var]=estimates_dict[var]
        if intercept:
            neighborhood_estimates_dict['Intercept_n']=estimates_dict['Intercept_n']
        
        #create df with correlates excluding neighborhood variables
        x= [item for item in x if item not in neighborhood_vars]
        head=x
        x_df=df[x]
        if intercept:
            head.insert(0,'Intercept')

        #remove contribution of neighborhood vars to yhat
        for col in neighborhood_vars:
            yhat-=neighbor_df[col]*estimates_dict[col]
        if intercept:
            yhat-=estimates_dict['Intercept_n']

        for i in range(n_bootstraps):
            #assign random resids
            bootstrap_sample['rb%s'%(i+1)] = np.random.choice(resids, n, replace=True)
            #add boostrap resids to fitted y values
            bootstrap_sample['yb%s'%(i+1)]=yhat+bootstrap_sample['rb%s'%(i+1)]
            
            #fit linear reg model using new y values
            res=fit_linear_reg(y_df=bootstrap_sample['yb%s'%(i+1)]
                ,x_df=x_df
                ,intercept=intercept)

            resids=res.resid
            bootstrap_estimates=pd.concat([bootstrap_estimates,(res.params.reset_index().transpose().iloc[1:])])
            print("Bootstrap sample %s completed"%(i+1))

    else:
        for i in range(n_bootstraps):
            #assign random resids
            bootstrap_sample['rb%s'%(i+1)] = np.random.choice(resids, n, replace=True)
            #add boostrap resids to fitted y values
            bootstrap_sample['yb%s'%(i+1)]=bootstrap_sample['yhat']+bootstrap_sample['rb%s'%(i+1)]
            
            #fit linear reg model using new y values
            res=fit_linear_reg(y_df=bootstrap_sample['yb%s'%(i+1)]
                ,x_df=x_df
                ,intercept=intercept)
            resids=res.resid
            bootstrap_estimates=pd.concat([bootstrap_estimates,(res.params.reset_index().transpose().iloc[1:])])
            
            print("Bootstrap sample %s completed"%(i+1))
    
    #compute percentiles for each parameter
    lower_bound=np.percentile(bootstrap_estimates, 2.5, axis=0, keepdims=False)
    upper_bound=np.percentile(bootstrap_estimates, 97.5, axis=0, keepdims=False)
    bootstrap_estimates.loc[n_bootstraps,:]=lower_bound
    bootstrap_estimates.loc[n_bootstraps+1,:]=upper_bound
    
    #export to csv
    print('Exporting bootstrap samples')
    bootstrap_sample.to_csv('%s%s%s.csv'%(method,y[0].replace('Death_1M','Death'),'_Int' if intercept==True else ''),index=False)
    print('Completed!')
    
    bootstrap_estimates.columns=head
    bootstrap_estimates.index=list(np.arange(n_bootstraps)+1)+['2.5th percentile','97.5th percentile']
    print('Exporting bootstrap estimates...')
    bootstrap_estimates.to_csv('%s%s%s_Estimates.csv'%(method,y[0].replace('Death_1M','Death'),'_Int' if intercept==True else ''),index_label='bootstrap')
    print('Completed!')
        

In [214]:
#run program
nonparm_bootstrapping(
    file_name='fulldataset (1).csv' #path or filename
    ,method='STEM' #SAR, STEM, OLS
    ,y=['PrevRate'] #PrevRate or Death_1M
    ,n_bootstraps=5 #number of bootstrap samples
    ,intercept=True #set to True or False
)

C:\Users\fongx\AppData\Local\Temp\ipykernel_18244\107877649.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  estimates['Significant Correlate']=estimates['Significant Correlate'].str.replace('*','').str.strip()


Bootstrap sample 1 completed
Bootstrap sample 2 completed
Bootstrap sample 3 completed
Bootstrap sample 4 completed
Bootstrap sample 5 completed
Exporting bootstrap samples
Completed!
Exporting bootstrap estimates...
Completed!


In [215]:
#run program
nonparm_bootstrapping(
    file_name='fulldataset (1).csv' #path or filename
    ,method='STEM' #SAR, STEM, OLS
    ,y=['PrevRate'] #PrevRate or Death_1M
    ,n_bootstraps=5 #number of bootstrap samples
    ,intercept=False #set to True or False
)

C:\Users\fongx\AppData\Local\Temp\ipykernel_18244\107877649.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  estimates['Significant Correlate']=estimates['Significant Correlate'].str.replace('*','').str.strip()


Bootstrap sample 1 completed
Bootstrap sample 2 completed
Bootstrap sample 3 completed
Bootstrap sample 4 completed
Bootstrap sample 5 completed
Exporting bootstrap samples


In [ ]:
nonparm_bootstrapping(
    file_name='fulldataset (1).csv'
    ,method='STEM' 
    ,y=['PrevRate']     
    ,n_bootstraps=1000 
    ,intercept=True 
)